In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import time

pd.set_option('display.max_columns', 50)

from evaluation_and_testing.testing import evaluate_train_test_split_explicit
from evaluation_and_testing.testing import evaluate_leave_one_out_explicit
from evaluation_and_testing.testing import evaluate_train_test_split_implicit
from evaluation_and_testing.testing import evaluate_leave_one_out_implicit

from recommenders.recommender import Recommender

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the data

In [3]:
ml_ratings_df = pd.read_csv(os.path.join("data", "movielens_small", "ratings.csv")).rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
ml_movies_df = pd.read_csv(os.path.join("data", "movielens_small", "movies.csv")).rename(columns={'movieId': 'item_id'})
ml_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')

display(ml_movies_df.head(10))

# Filter the data to reduce the number of movies
seed = 6789
rng = np.random.RandomState(seed=seed)
left_ids = rng.choice(ml_movies_df['item_id'], size=1000, replace=False)

ml_ratings_df = ml_ratings_df.loc[ml_ratings_df['item_id'].isin(left_ids)]
ml_movies_df = ml_movies_df.loc[ml_movies_df['item_id'].isin(left_ids)]
ml_df = ml_df.loc[ml_df['item_id'].isin(left_ids)]

print("Number of chosen interactions: {}".format(len(ml_ratings_df)))

display(ml_df.head(10))

,item_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Number of chosen interactions: 9692


,user_id,item_id,rating,timestamp,title,genres
572,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
573,5,50,4.0,847434881,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
574,6,50,1.0,845553381,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
575,7,50,4.5,1106635993,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
576,8,50,5.0,839463644,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
577,16,50,4.0,1377476781,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
578,17,50,4.5,1305697013,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
579,18,50,5.0,1455049343,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
580,23,50,4.0,1107163741,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
581,24,50,4.0,1458942023,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


# Baseline recommenders

**Task 1.** Implement the MostPopularRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [4]:
class MostPopularRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.most_popular_movies = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
#         display(interactions_df)
        movies_id_df = interactions_df.loc[:, ["item_id", "user_id"]]       
        movies_watched_count = movies_id_df.groupby('item_id').count()
        movies_watched_count = movies_watched_count.sort_values(by='user_id', ascending=False)
        self.most_popular_movies = movies_watched_count.index[:10]
        
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        #score = 1 / pozycja
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({
                'user_id': user['user_id'],
                'item_id': self.most_popular_movies[:n_recommendations],
                'score': [1 / (i + 1) for i in range(n_recommendations)] 
            })

            recommendations = pd.concat([recommendations, user_recommendations])
        
        return recommendations       
    
# Quick test of the recommender

most_popular_recommender = MostPopularRecommender()
most_popular_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = most_popular_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

most_popular_recommender = MostPopularRecommender()

most_popular_results = [['MostPopularRecommender'] + list(evaluate_leave_one_out_implicit(
    most_popular_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

most_popular_results = pd.DataFrame(
    most_popular_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(most_popular_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,50,1.000000,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
1,1,780,0.500000,Independence Day (a.k.a. ID4) (1996),Action|Adventure|Sci-Fi|Thriller
2,1,858,0.333333,"Godfather, The (1972)",Crime|Drama
3,1,595,0.250000,Beauty and the Beast (1991),Animation|Children|Fantasy|Musical|Romance|IMAX
4,1,1704,0.200000,Good Will Hunting (1997),Drama|Romance
5,1,1213,0.166667,Goodfellas (1990),Crime|Drama
6,1,2716,0.142857,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi
7,1,586,0.125000,Home Alone (1990),Children|Comedy
8,1,208,0.111111,Waterworld (1995),Action|Adventure|Sci-Fi
9,1,7438,0.100000,Kill Bill: Vol. 2 (2004),Action|Drama|Thriller


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,MostPopularRecommender,0.026667,0.05,0.083333,0.163333,0.026667,0.040079,0.053997,0.079749


**Task 2.** Implement the HighestRatedRecommender (check the slides for class 1), but recommend only those movies which got at least 50 ratings. Evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [5]:
class HighestRatedRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.movies_mean_rating = None
        self.movie = None
        self.rating = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """        
        movies_ratings = interactions_df.loc[:, ['item_id', 'rating']]        
        movies_count_rating = movies_ratings.groupby('item_id').count()
        self.movies_mean_rating = movies_ratings.groupby('item_id').mean()
        self.movies_mean_rating = self.movies_mean_rating.loc[movies_count_rating['rating'] >= 50]
        self.movies_mean_rating = self.movies_mean_rating.sort_values('rating', ascending=False)

    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
#         score = rating / 5
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({
                'user_id': user['user_id'],
                'item_id': self.movies_mean_rating.index[:n_recommendations],
                'score': self.movies_mean_rating['rating'][:n_recommendations] / 5
            })

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations
    
# Quick test of the recommender

highest_rated_recommender = HighestRatedRecommender()
highest_rated_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = highest_rated_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

highest_rated_recommender = HighestRatedRecommender()

highest_rated_results = [['HighestRatedRecommender'] + list(evaluate_leave_one_out_implicit(
    highest_rated_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=6789))]

highest_rated_results = pd.DataFrame(
    highest_rated_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(highest_rated_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,858,0.857812,"Godfather, The (1972)",Crime|Drama
1,1,750,0.853608,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Comedy|War
2,1,1213,0.850000,Goodfellas (1990),Crime|Drama
3,1,50,0.847549,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
4,1,1225,0.836842,Amadeus (1984),Drama
5,1,1203,0.829825,12 Angry Men (1957),Drama
6,1,2502,0.818085,Office Space (1999),Comedy|Crime
7,1,1704,0.815603,Good Will Hunting (1997),Drama|Romance
8,1,2692,0.800000,Run Lola Run (Lola rennt) (1998),Action|Crime
9,1,6502,0.794828,28 Days Later (2002),Action|Horror|Sci-Fi


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,HighestRatedRecommender,0.013333,0.04,0.073333,0.113333,0.013333,0.02754,0.041603,0.054395


**Task 3.** Implement the RandomRecommender (check the slides for class 1), evaluate it with leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [6]:
class RandomRecommender(Recommender):
    """
    Base recommender class.
    """
    
    def __init__(self, seed=0):
        """
        Initialize base recommender params and variables.
        
        :param int seed: Seed for the random number generator.
        """
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
        # Write your code here
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        # Write your code here
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        movies = items_df['item_id'].unique().tolist()
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_recommendations = pd.DataFrame({
                'user_id': user['user_id'],
                'item_id': self.rng.choice(movies, n_recommendations, replace=False),
                'score': 1.0
            })
            
            recommendations = pd.concat([recommendations, user_recommendations])
            
        recommendations = recommendations.reset_index(drop=True)

        return recommendations
    
# Quick test of the recommender

random_recommender = RandomRecommender(seed=6789)
random_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = random_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = recommendations.merge(ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

random_recommender = RandomRecommender(seed=seed)

random_results = [['RandomRecommender'] + list(evaluate_leave_one_out_implicit(
    random_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, max_evals=300, seed=seed))]

random_results = pd.DataFrame(
    random_results, 
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(HTML(random_results.to_html()))

Recommendations


,user_id,item_id,score,title,genres
0,1,472,1.0,I'll Do Anything (1994),Comedy|Drama
1,1,26631,1.0,Alice (Neco z Alenky) (1988),Animation|Fantasy|Mystery
2,1,63276,1.0,Crows Zero (Kurôzu zero) (2007),Action
3,1,2815,1.0,Iron Eagle (1986),Action|War
4,1,31553,1.0,Double Dragon (1994),Action|Adventure|Sci-Fi
5,1,46335,1.0,"Fast and the Furious: Tokyo Drift, The (Fast and the Furious 3, The) (2006)",Action|Crime|Drama|Thriller
6,1,1107,1.0,Loser (1991),Comedy
7,1,42728,1.0,Tristan & Isolde (2006),Drama|Romance
8,1,4558,1.0,Twins (1988),Comedy
9,1,77866,1.0,Robin Hood (2010),Action|Adventure|Drama|Romance|War


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomRecommender,0.0,0.003333,0.01,0.013333,0.0,0.002103,0.004828,0.00588


# Linear Regression Recommender

For every movie we transform its genres into one-hot encoded features and we normalize them, for every user we count percentages for all genres how often do they appear among films watched by the user, we multiply both vectors (for the item and the user) to obtain explanatory variables, and then we fit a linear regression model to those features and actual ratings.

**Task 4.** Implement the calculate_user_genres method for calculating a DataFrame with one row per user and columns corresponding to genres (e.g. 'user_action', 'user_drama') with values calculated as follows:

- count the number of times a given user watched a given genre,
- apply a natural logarithm to this value plus one,
- normalize those values so that the sum of them over all columns within a row is equal to 1.

Implement the calculate_item_genres method for replacing the 'genres' column with one column per genre (e.g. 'action', 'drama') with values calculated as follows:

- place 1 in every column for which the genre appears in genres,
- normalize those values so that the sum of them over all genre columns within a row is equal to 1.

If item_features is None, then first find all genres and prepare a list of them. If item_features is not None, then create columns based on this list. Return both the transformed DataFrame and the list of genres.  Do not use MultiLabelBinarizer.

Note that in this second method you have to preserve the remaining structure of the DataFrame.

Evaluate the LinearRegressionRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [36]:
# Prepare data for tests
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")
display(interactions_df)

,user_id,item_id,rating,timestamp,title,genres
0,1,50,5.0,964982931,"Usual Suspects, The (1995)","[crime, mystery, thriller]"
1,5,50,4.0,847434881,"Usual Suspects, The (1995)","[crime, mystery, thriller]"
2,6,50,1.0,845553381,"Usual Suspects, The (1995)","[crime, mystery, thriller]"
3,7,50,4.5,1106635993,"Usual Suspects, The (1995)","[crime, mystery, thriller]"
4,8,50,5.0,839463644,"Usual Suspects, The (1995)","[crime, mystery, thriller]"
...,...,...,...,...,...,...
9687,610,110746,4.0,1493850133,Hatchet III (2013),"[comedy, horror]"
9688,610,113159,3.5,1493848692,Life After Beth (2014),"[comedy, horror, romance]"
9689,610,114707,3.0,1493848404,Horns (2014),"[horror, mystery]"
9690,610,130840,4.5,1493848621,Spring (2015),"[horror, romance, sci_fi]"


In [210]:
def calculate_user_genres(interactions_df):
    users_df = interactions_df.copy()
    users_df = users_df.explode('genres')
    users_df['val'] = 1
    users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count').fillna(0)
    #users_df = users_df.apply(lambda x: np.log(x+1))
    users_df = users_df.div(users_df.sum(axis=1), axis=0)
    users_df = users_df.rename_axis(None, axis=1)
    users_df = users_df.add_prefix('user_')
#     users_df = interactions_df[['user_id', 'genres']].copy()
#     users_df = users_df.explode('genres')
#     users_df['val'] = 1
#     users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
#     users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
#     users_df = users_df.rename_axis(None, axis=1).fillna(0)
#     users_df = users_df.add_prefix('user_')
    return users_df
# calculate_user_genres(interactions_df)

In [211]:
# Test the calculate_user_genres method
display(calculate_user_genres(interactions_df))

,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western
user_id,,,,,,,,,,,,,,,,,,,,
1,0.149254,0.119403,0.044776,0.074627,0.119403,0.089552,0.000000,0.059701,0.044776,0.000000,0.000000,0.000000,0.014925,0.014925,0.0,0.074627,0.059701,0.089552,0.029851,0.014925
2,0.125000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.062500,0.062500,0.187500,0.000000,0.062500
3,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.000000,0.000000,0.000000,0.0,0.000000,0.285714,0.142857,0.000000,0.000000
4,0.017857,0.017857,0.035714,0.053571,0.196429,0.089286,0.000000,0.267857,0.035714,0.017857,0.000000,0.017857,0.053571,0.000000,0.0,0.125000,0.000000,0.035714,0.017857,0.017857
5,0.000000,0.000000,0.058824,0.058824,0.058824,0.058824,0.000000,0.176471,0.058824,0.000000,0.000000,0.058824,0.058824,0.058824,0.0,0.176471,0.000000,0.058824,0.058824,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.048889,0.040000,0.008889,0.022222,0.164444,0.066667,0.004444,0.297778,0.044444,0.004444,0.013333,0.008889,0.004444,0.022222,0.0,0.124444,0.031111,0.066667,0.022222,0.004444
607,0.148936,0.063830,0.000000,0.021277,0.127660,0.085106,0.000000,0.212766,0.021277,0.000000,0.063830,0.000000,0.021277,0.021277,0.0,0.042553,0.085106,0.063830,0.021277,0.000000
608,0.120172,0.055794,0.021459,0.064378,0.184549,0.077253,0.000000,0.107296,0.055794,0.000000,0.051502,0.008584,0.008584,0.017167,0.0,0.042918,0.077253,0.098712,0.008584,0.000000


In [163]:
from sklearn.preprocessing import MultiLabelBinarizer

def calculate_item_genres(item_genres, item_features=None):
    item_genres = item_genres.copy()
    item_genres = item_genres.explode('genres')
    item_genres['value'] = 1
    if item_features is None:
        item_features = item_genres['genres'].values
        item_features = np.unique(item_features).tolist()
    item_genres_pivot = item_genres.pivot_table(index='item_id',columns='genres',values='value').fillna(0)
    item_genres_pivot = item_genres_pivot.reindex(item_features,axis=1).fillna(0)
    item_genres_pivot = item_genres_pivot.div(item_genres_pivot.sum(axis=1), axis=0)
    item_genres = item_genres.merge(item_genres_pivot, on='item_id')
    item_genres = item_genres.drop(columns=['value'])
    return item_genres, item_features


In [164]:
# Test the method

item_genres, item_features = calculate_item_genres(interactions_df)
display(item_genres)
print(item_features)

item_genres, item_features = calculate_item_genres(interactions_df, ['comedy', 'crime', 'drama', 'horror', 'test_category'])
display(item_genres)
print(item_features)

,user_id,item_id,rating,timestamp,title,genres,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film_noir,horror,imax,musical,mystery,no_genres_listed,romance,sci_fi,thriller,war,western
0,1,50,5.0,964982931,"Usual Suspects, The (1995)",crime,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
1,1,50,5.0,964982931,"Usual Suspects, The (1995)",mystery,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
2,1,50,5.0,964982931,"Usual Suspects, The (1995)",thriller,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
3,5,50,4.0,847434881,"Usual Suspects, The (1995)",crime,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
4,5,50,4.0,847434881,"Usual Suspects, The (1995)",mystery,0.0,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25601,610,130840,4.5,1493848621,Spring (2015),romance,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.333333,0.333333,0.000000,0.0,0.0
25602,610,130840,4.5,1493848621,Spring (2015),sci_fi,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.333333,0.333333,0.000000,0.0,0.0
25603,610,149011,3.5,1493848843,He Never Died (2015),comedy,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0
25604,610,149011,3.5,1493848843,He Never Died (2015),drama,0.0,0.0,0.0,0.0,0.333333,0.000000,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0


['action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film_noir', 'horror', 'imax', 'musical', 'mystery', 'no_genres_listed', 'romance', 'sci_fi', 'thriller', 'war', 'western']


,user_id,item_id,rating,timestamp,title,genres,comedy,crime,drama,horror,test_category
0,1,50,5.0,964982931,"Usual Suspects, The (1995)",crime,0.000000,1.0,0.000000,0.000000,0.0
1,1,50,5.0,964982931,"Usual Suspects, The (1995)",mystery,0.000000,1.0,0.000000,0.000000,0.0
2,1,50,5.0,964982931,"Usual Suspects, The (1995)",thriller,0.000000,1.0,0.000000,0.000000,0.0
3,5,50,4.0,847434881,"Usual Suspects, The (1995)",crime,0.000000,1.0,0.000000,0.000000,0.0
4,5,50,4.0,847434881,"Usual Suspects, The (1995)",mystery,0.000000,1.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
25601,610,130840,4.5,1493848621,Spring (2015),romance,0.000000,0.0,0.000000,1.000000,0.0
25602,610,130840,4.5,1493848621,Spring (2015),sci_fi,0.000000,0.0,0.000000,1.000000,0.0
25603,610,149011,3.5,1493848843,He Never Died (2015),comedy,0.333333,0.0,0.333333,0.333333,0.0
25604,610,149011,3.5,1493848843,He Never Died (2015),drama,0.333333,0.0,0.333333,0.333333,0.0


['comedy', 'crime', 'drama', 'horror', 'test_category']


In [212]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer

from recommenders.recommender import Recommender

class LinearRegressionRecommender(Recommender):
    """
    Linear regression recommender class.
    """
    
    def __init__(self):
        """
        Initialize recommender params and variables.
        """
        self.model = None
        self.mlb = None
        self.users_dict = None
        self.user_features = None
        
        self.uses_dot_product = True
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        # Transform genres to a more code-friendly form
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df = self._transform_genres(interactions_df)
        
        # Prepare user features
        
        users_df = interactions_df[['user_id', 'genres']].copy()
        users_df = users_df.explode('genres')
        users_df['val'] = 1
        users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
        users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
        users_df = users_df.rename_axis(None, axis=1).fillna(0)
        users_df = users_df.add_prefix('user_')
#         display(users_df.head(10))
        
        self.users_dict = users_df.to_dict('index')
        
        self.user_features = users_df.columns.tolist()
        
        interactions_df = interactions_df.merge(users_df, on='user_id')
#         display(interactions_df.head(10))
                
        # Prepare item features
        
        # Transform genres into binary values
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
        # Normalize the values so that each movie's genres sum up to 1
        
        interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
            / interactions_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)
        
#         display(interactions_df.loc[:, self.mlb.classes_].head(10))

        # Prepare input data and fit the model
    
        # Dot product for personalization
        if self.uses_dot_product:
            interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
                * interactions_df[self.user_features].values
    
#         display(interactions_df.head(10))
        
        if self.uses_dot_product:  # Personalized
            x = interactions_df.loc[:, self.mlb.classes_].values
        else:  # Non-personalized
            x = interactions_df.loc[:, list(self.mlb.classes_) + self.user_features].values
        display(x)
        y = interactions_df['rating'].values
    
        self.model = LinearRegression().fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Transform the item to be scored into proper features
        
        items_df = items_df.copy()
        items_df = self._transform_genres(items_df)
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
        items_df[self.mlb.classes_] = items_df[self.mlb.classes_] \
            / items_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)

        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = pd.DataFrame.from_dict(
                    {user['user_id']: [1 / len(self.user_features)]*len(self.user_features)}, orient='index')
                user_df.columns = self.user_features
#             display(user_df)
#             display(items_df)
            input_df = items_df.copy()
            
            if self.uses_dot_product:
                input_df[self.mlb.classes_] = items_df[self.mlb.classes_] * user_df.values
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, self.mlb.classes_].values)
            else:
                input_df = input_df.merge(user_df, how='cross')
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, list(self.mlb.classes_) + self.user_features].values)
    
            chosen_pos = np.argsort(-scores)[:n_recommendations]
        
            user_recommendations = []
            for item_pos in chosen_pos:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_pos]['item_id'],
                        'score': scores[item_pos]
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations
    
    def _transform_genres(self, df):
        """
        Transforms a string with genres into a list of cleaned genre names.
        
        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df

In [213]:
# Quick test of the recommender

lr_recommender = LinearRegressionRecommender()
lr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = lr_recommender.recommend(pd.DataFrame([[3], [5], [39]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
display(recommendations)

array([[0.        , 0.        , 0.        , ..., 0.02985075, 0.        ,
        0.        ],
       [0.07462687, 0.        , 0.        , ..., 0.04477612, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02985075, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.125     , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

,user_id,item_id,score,title,genres
0,3,51562,3.501376,Babylon 5: The Gathering (1993),Sci-Fi
1,3,5468,3.501376,20 Million Miles to Earth (1957),Sci-Fi
2,3,176371,3.501376,Blade Runner 2049 (2017),Sci-Fi
3,3,147384,3.501376,Doctor Who: The Runaway Bride (2007),Sci-Fi
4,3,2698,3.501376,Zone 39 (1997),Sci-Fi
5,3,2661,3.501376,It Came from Outer Space (1953),Sci-Fi
6,3,2311,3.501376,2010: The Year We Make Contact (1984),Sci-Fi
7,3,3926,3.425357,Voyage to the Bottom of the Sea (1961),Adventure|Sci-Fi
8,3,4953,3.425357,"People That Time Forgot, The (1977)",Adventure|Sci-Fi
9,3,3994,3.425357,Unbreakable (2000),Drama|Sci-Fi


### Train-test split test

In [98]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_train_test_split_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.016594,0.348461,0.230529


Total evaluation time: 34.282792806625366


### Leave-one-out test

In [117]:
lr_recommender = LinearRegressionRecommender()

t0 = time.time()

results = [['LinearRegressionRecommender'] + list(evaluate_leave_one_out_explicit(
    lr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,RMSE,MRE,TRE
0,LinearRegressionRecommender,1.045415,0.401399,0.242301


Total evaluation time: 55.69004774093628


# TF-IDF Recommender
TF-IDF stands for term frequency–inverse document frequency. Typically Tf-IDF method is used to assign keywords (words describing the gist of a document) to documents in a corpus of documents.

In our case we will treat users as documents and genres as words.

Term-frequency is given by the following formula:
<center>
$$
    \text{tf}(g, u) = f_{g, u}
$$
</center>
where $f_{g, i}$ is the number of times genre $g$ appear for movies watched by user $u$.

Inverse document frequency is defined as follows:
<center>
$$
    \text{idf}(g) = \log \frac{N}{n_g}
$$
</center>
where $N$ is the number of users and $n_g$ is the number of users with $g$ in their genres list.

Finally, tf-idf is defined as follows:
<center>
$$
    \text{tfidf}(g, u) = \text{tf}(g, u) \cdot \text{idf}(g)
$$
</center>

In our case we will measure how often a given genre appears for movies watched by a given user vs how often it appears for all users. To obtain a movie score we will take the average of its genres' scores for this user.

**Task 5.** Implement the following method for calculating a TF-IDF scores in a form of a dict (use defaultdict):

`{(1, 'action'): 0.45306430692185395, (1, 'adventure'): 0.39370003643934415, (1, 'animation'): 0.20886242957049514, ...}`

without using TfidfVectorizer (you can use loops).

Evaluate the TFIDFRecommender with it using leave-one-out procedure for implicit feedback, print HR@1, HR@3, HR@5, HR@10, NDCG@1, NDCG@3, NDCG@5, NDCG@10.

In [195]:
def calculate_tf_idf_scores(interactions_df):
    interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.split("|")
    interactions_df = interactions_df.explode('genres')
    interactions_df['value'] = 1
    users = interactions_df['user_id'].values
    users = np.unique(users).tolist()
    no_users = len(users)
    genres = interactions_df['genres'].values
    genres = np.unique(genres).tolist()
    interactions_df = interactions_df.pivot_table(index='user_id', columns='genres', values='value', aggfunc='count').fillna(0)
    interactions_df = interactions_df.rename_axis(None, axis=1)
    users_dict = interactions_df.to_dict('index')
    genres_idf = defaultdict()
    for genre in genres:
        ng = np.count_nonzero(interactions_df[genre])
        genres_idf[genre] = np.log(no_users/ng)
    tfidf_scores = defaultdict(lambda:0.0)
    for user in users:
        for genre in genres:
            tfidf_scores[user, genre] = users_dict[user][genre] * genres_idf[genre]
#     display(tfidf_scores)
    return tfidf_scores
# calculate_tf_idf_scores(interactions_df)

In [196]:
# Test the method
interactions_df = pd.merge(ml_ratings_df, ml_movies_df, on='item_id')
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("(", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(")", "", regex=False)
interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
# display(interactions_df)
tfidf_scores = calculate_tf_idf_scores(interactions_df)
display(tfidf_scores)
print(round(tfidf_scores[(1, "action")], 3))
print(round(tfidf_scores[(55, "comedy")], 3))

defaultdict(<function __main__.calculate_tf_idf_scores.<locals>.<lambda>()>,
            {(1, 'action'): 2.198046500486951,
             (1, 'adventure'): 2.6013213256521714,
             (1, 'animation'): 2.628413572457773,
             (1, 'children'): 3.1107481541030797,
             (1, 'comedy'): 1.791840171673406,
             (1, 'crime'): 2.3086654110482208,
             (1, 'documentary'): 0.0,
             (1, 'drama'): 0.6840579435170525,
             (1, 'fantasy'): 1.6501390107578158,
             (1, 'film_noir'): 0.0,
             (1, 'horror'): 0.0,
             (1, 'imax'): 0.0,
             (1, 'musical'): 0.9297589489250216,
             (1, 'mystery'): 0.6473875634104785,
             (1, 'no_genres_listed'): 0.0,
             (1, 'romance'): 1.5340801351916244,
             (1, 'sci_fi'): 1.2729788914477906,
             (1, 'thriller'): 1.7060590271175973,
             (1, 'war'): 2.121743921370525,
             (1, 'western'): 1.1786549963416462,
             (2,

2.198
0.224


In [197]:
class TFIDFRecommender(Recommender):
    """
    Recommender based on the TF-IDF method.
    """
    
    def __init__(self):
        """
        Initialize base recommender params and variables.
        """
        self.tfidf_scores = None
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        self.tfidf_scores = defaultdict(lambda: 0.0)

        # Prepare the corpus for tfidf calculation
        
        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace("-", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.replace(" ", "_", regex=False)
        interactions_df.loc[:, 'genres'] = interactions_df['genres'].str.lower()
                
        self.tfidf_scores = calculate_tf_idf_scores(interactions_df)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # Transform genres to a unified form used by the vectorizer
        
        items_df = items_df.copy()
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace("-", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.replace(" ", "_", regex=False)
        items_df.loc[:, 'genres'] = items_df['genres'].str.lower()
        items_df.loc[:, 'genres'] = items_df['genres'].str.split("|")
                
        # Score items    
        
        for uix, user in users_df.iterrows():
            items = []
            for iix, item in items_df.iterrows():
                score = 0.0
                for genre in item['genres']:
                    score += self.tfidf_scores[(user['user_id'], genre)]
                score /= len(item['genres'])
                items.append((item['item_id'], score))
                
            items = sorted(items, key=lambda x: x[1], reverse=True)
            user_recommendations = pd.DataFrame({'user_id': user['user_id'],
                                                 'item_id': [item[0] for item in items][:n_recommendations],
                                                 'score': [item[1] for item in items][:n_recommendations]})

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

In [198]:
# Print movies watched by user 3, 5, 39

active_user_movies = ml_df.loc[(ml_df['user_id'] == 3) | (ml_df['user_id'] == 5) | (ml_df['user_id'] == 39)]
print("Active users history")
display(active_user_movies)

Active users history


,user_id,item_id,rating,timestamp,title,genres
573,5,50,4.0,847434881,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
588,39,50,5.0,974788030,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
7762,39,1213,4.0,974789540,Goodfellas (1990),Crime|Drama
9403,39,1500,4.0,974789727,Grosse Pointe Blank (1997),Comedy|Crime|Romance
11108,39,2078,3.0,974788976,"Jungle Book, The (1967)",Animation|Children|Comedy|Musical
13425,39,2692,5.0,974789111,Run Lola Run (Lola rennt) (1998),Action|Crime
14203,39,2947,3.0,974789327,Goldfinger (1964),Action|Adventure|Thriller
18810,3,5181,5.0,1306463718,Hangar 18 (1980),Action|Sci-Fi|Thriller
18827,3,7991,5.0,1306463684,Death Race 2000 (1975),Action|Sci-Fi
18832,3,70946,5.0,1306463815,Troll 2 (1990),Fantasy|Horror


In [199]:
# Quick test of the recommender

tfidf_recommender = TFIDFRecommender()
tfidf_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = tfidf_recommender.recommend(pd.DataFrame([[3], [5], [39]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(recommendations)

Recommendations


,user_id,item_id,score,title,genres
0,3,1322,1.075471,Amityville 1992: It's About Time (1992),Horror
1,3,1976,1.075471,Friday the 13th Part 3: 3D (1982),Horror
2,3,1980,1.075471,Friday the 13th Part VII: The New Blood (1988),Horror
3,3,2517,1.075471,Christine (1983),Horror
4,3,2878,1.075471,Hell Night (1981),Horror
5,3,3017,1.075471,Creepshow 2 (1987),Horror
6,3,3021,1.075471,"Funhouse, The (1981)",Horror
7,3,3652,1.075471,"City of the Living Dead (a.k.a. Gates of Hell,...",Horror
8,3,3839,1.075471,Phantasm IV: Oblivion (1998),Horror
9,3,3908,1.075471,Urban Legends: Final Cut (2000),Horror


### Train-test split test

In [200]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_train_test_split_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,TFIDFRecommender,0.013423,0.029083,0.042506,0.082774,0.013423,0.022717,0.028204,0.041327


Total evaluation time: 48.94744253158569


### Leave-one-out test

In [201]:
tfidf_recommender = TFIDFRecommender()

t0 = time.time()

results = [['TFIDFRecommender'] + list(evaluate_leave_one_out_implicit(
    tfidf_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id']], ml_movies_df, max_evals=300, seed=6789))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(results)

print('Total evaluation time: {}'.format(time.time() - t0))

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,TFIDFRecommender,0.0,0.006667,0.013333,0.026667,0.0,0.004206,0.007077,0.011674


Total evaluation time: 55.12540793418884


**Task 6\*.** Implement an SVRRecommender by replacing the LinearRegression model with the SVR model (`from sklearn.svm import SVR`). Use separate features for users and items, not the dot product.

Tune params of the SVR model to obtain as good results as you can. 

To do tuning properly:

- divide the set into training, validation and test sets (randomly divide the dataset in proportions 60%-20%-20%),
- train the model with different sets of tunable parameters on the training set, 
- choose the best tunable params based on results on the validation set, 
- provide the final evaluation metrics on the test set for the best model obtained during tuning.

Recommended method of tuning: use hyperopt. Install the package using the following command: `pip install hyperopt`
    
Print the RMSE, MAPE and TRE on the test set. Use seed 6789.

In [206]:
from sklearn.svm import SVR
from sklearn.preprocessing import MultiLabelBinarizer

    
class SVRRecommender(Recommender):
    """
    SVR recommender class.
    """

    def __init__(self, kernel='rbf', c=1.0, epsilon=0.1):
        """
        Initialize base recommender params and variables.
        """
        super().__init__()
        self.model = None
        self.mlb = None
        self.kernel = kernel
        self.c = c
        self.epsilon = epsilon
        self.mlb = None
        self.users_dict = None
        self.user_features = None
        self.uses_dot_product = True

    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.

        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id
            and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id
            and the item feature columns.
        """

        interactions_df = pd.merge(interactions_df, items_df, on='item_id')
        interactions_df = self._transform_genres(interactions_df)
#         display(interactions_df)
        # Prepare user features
        
        users_df = interactions_df[['user_id', 'genres']].copy()
        users_df = users_df.explode('genres')
        users_df['val'] = 1
        users_df = users_df.pivot_table(index='user_id', columns='genres', values='val', aggfunc='count')
        users_df = users_df / users_df.sum(axis=1).values.reshape(-1, 1)
        users_df = users_df.rename_axis(None, axis=1).fillna(0)
        users_df = users_df.add_prefix('user_')
#         display(users_df.head(10))
        
        self.users_dict = users_df.to_dict('index')
        
        self.user_features = users_df.columns.tolist()
        
        interactions_df = interactions_df.merge(users_df, on='user_id')
#         display(interactions_df.head(10))
                
        # Prepare item features
        
        # Transform genres into binary values
        
        self.mlb = MultiLabelBinarizer()
        interactions_df = interactions_df.join(
            pd.DataFrame(self.mlb.fit_transform(interactions_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=interactions_df.index))
        
        # Normalize the values so that each movie's genres sum up to 1
        
        interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
            / interactions_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)
        display(interactions_df)
#         display(interactions_df.loc[:, self.mlb.classes_].head(10))

        # Prepare input data and fit the model
    
        # Dot product for personalization
        if self.uses_dot_product:
            interactions_df[self.mlb.classes_] = interactions_df[self.mlb.classes_] \
                * interactions_df[self.user_features].values
    
#         display(interactions_df.head(10))
        
        if self.uses_dot_product:  # Personalized
            x = interactions_df.loc[:, self.mlb.classes_].values
        else:  # Non-personalized
            x = interactions_df.loc[:, list(self.mlb.classes_) + self.user_features].values
#         print("x:")
#         display(x)
#         print("y:")
        
        y = interactions_df['rating'].values
#         display(y)
        svr = SVR(kernel=self.kernel, C=self.c, gamma=0.1, epsilon=self.epsilon)
        self.model = svr.fit(x, y)

    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns
        top n_recommendations for each user.

        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations
            should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations
            for each user.
        :rtype: pd.DataFrame
        """

        items_df = items_df.copy()
        items_df = self._transform_genres(items_df)
        
        items_df = items_df.join(
            pd.DataFrame(self.mlb.transform(items_df.pop('genres')),
                         columns=self.mlb.classes_,
                         index=items_df.index))
        
        items_df[self.mlb.classes_] = items_df[self.mlb.classes_] \
            / items_df[self.mlb.classes_].sum(axis=1).values.reshape(-1, 1)

        # Score the item
    
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            if user['user_id'] in self.users_dict:
                user_df = pd.DataFrame.from_dict({user['user_id']: self.users_dict[user['user_id']]}, orient='index')
            else:
                user_df = pd.DataFrame.from_dict(
                    {user['user_id']: [1 / len(self.user_features)]*len(self.user_features)}, orient='index')
                user_df.columns = self.user_features
#             display(user_df)
#             display(items_df)
            input_df = items_df.copy()
            
            if self.uses_dot_product:
                input_df[self.mlb.classes_] = items_df[self.mlb.classes_] * user_df.values
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, self.mlb.classes_].values)
            else:
                input_df = input_df.merge(user_df, how='cross')
#                 display(input_df)
                scores = self.model.predict(input_df.loc[:, list(self.mlb.classes_) + self.user_features].values)
    
            chosen_pos = np.argsort(-scores)[:n_recommendations]
        
            user_recommendations = []
            for item_pos in chosen_pos:
                user_recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_pos]['item_id'],
                        'score': scores[item_pos]
                    }
                )
                
            user_recommendations = pd.DataFrame(user_recommendations)

            recommendations = pd.concat([recommendations, user_recommendations])

        return recommendations

    @staticmethod
    def _transform_genres(df):
        """
        Transforms a string with genres into a list of cleaned genre names.

        :param pd.DataFrame df: A DataFrame with 'genres' column.
        """
        df.loc[:, 'genres'] = df['genres'].str.replace("-", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(" ", "_", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace("(", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.replace(")", "", regex=False)
        df.loc[:, 'genres'] = df['genres'].str.lower()
        df.loc[:, 'genres'] = df['genres'].str.split("|")
        return df
    
    
# Quick test of the recommender

svr_recommender = SVRRecommender()
svr_recommender.fit(ml_ratings_df, None, ml_movies_df)
recommendations = svr_recommender.recommend(pd.DataFrame([[1], [2], [6]], columns=['user_id']), ml_movies_df, 10)

recommendations = pd.merge(recommendations, ml_movies_df, on='item_id', how='left')
print("Recommendations")
display(HTML(recommendations.to_html()))

,user_id,item_id,rating,timestamp,title,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film_noir,horror,imax,musical,mystery,no_genres_listed,romance,sci_fi,thriller,war,western
0,1,50,5.0,964982931,"Usual Suspects, The (1995)",0.149254,0.119403,0.044776,0.074627,0.119403,0.089552,0.0,0.059701,0.044776,0.0,0.0,0.0,0.014925,0.014925,0.0,0.074627,0.059701,0.089552,0.029851,0.014925,0.000000,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.00,0.333333,0.0,0.0
1,1,423,3.0,964982363,Blown Away (1994),0.149254,0.119403,0.044776,0.074627,0.119403,0.089552,0.0,0.059701,0.044776,0.0,0.0,0.0,0.014925,0.014925,0.0,0.074627,0.059701,0.089552,0.029851,0.014925,0.500000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.500000,0.0,0.0
2,1,543,4.0,964981179,So I Married an Axe Murderer (1993),0.149254,0.119403,0.044776,0.074627,0.119403,0.089552,0.0,0.059701,0.044776,0.0,0.0,0.0,0.014925,0.014925,0.0,0.074627,0.059701,0.089552,0.029851,0.014925,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.00,0.333333,0.0,0.0
3,1,780,3.0,964984086,Independence Day (a.k.a. ID4) (1996),0.149254,0.119403,0.044776,0.074627,0.119403,0.089552,0.0,0.059701,0.044776,0.0,0.0,0.0,0.014925,0.014925,0.0,0.074627,0.059701,0.089552,0.029851,0.014925,0.250000,0.250000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.25,0.250000,0.0,0.0
4,1,940,5.0,964982176,"Adventures of Robin Hood, The (1938)",0.149254,0.119403,0.044776,0.074627,0.119403,0.089552,0.0,0.059701,0.044776,0.0,0.0,0.0,0.014925,0.014925,0.0,0.074627,0.059701,0.089552,0.029851,0.014925,0.333333,0.333333,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.00,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9687,481,1465,4.0,884773704,Rosewood (1997),0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0
9688,175,44613,5.0,1234189390,Take the Lead (2006),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.666667,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.0
9689,175,47382,5.0,1234189512,Step Up (2006),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.666667,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.500000,0.00,0.000000,0.0,0.0
9690,578,6751,2.5,1300990921,Cold Creek Manor (2003),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.250000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.500000,0.0,0.0


Recommendations


,user_id,item_id,score,title,genres
0,1,4239,3.800866,Blow (2001),Crime|Drama
1,1,55272,3.800866,We Own the Night (2007),Crime|Drama
2,1,37741,3.800866,Capote (2005),Crime|Drama
3,1,39292,3.800866,"Good Night, and Good Luck. (2005)",Crime|Drama
4,1,31309,3.800866,Rocco and His Brothers (Rocco e i suoi fratelli) (1960),Crime|Drama
5,1,5954,3.800866,25th Hour (2002),Crime|Drama
6,1,4139,3.800866,No Man's Land (1987),Crime|Drama
7,1,4889,3.800866,Heist (2001),Crime|Drama
8,1,5888,3.800866,Brother (Brat) (1997),Crime|Drama
9,1,858,3.800866,"Godfather, The (1972)",Crime|Drama


In [207]:
svr_recommender = SVRRecommender(kernel='rbf', c=100, epsilon=0.1)

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, ml_ratings_df.loc[:, ['user_id', 'item_id', 'rating']], ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(results)

,user_id,item_id,rating,title,user_action,user_adventure,user_animation,user_children,user_comedy,user_crime,user_documentary,user_drama,user_fantasy,user_film_noir,user_horror,user_imax,user_musical,user_mystery,user_no_genres_listed,user_romance,user_sci_fi,user_thriller,user_war,user_western,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film_noir,horror,imax,musical,mystery,no_genres_listed,romance,sci_fi,thriller,war,western
0,288,2311,3.0,2010: The Year We Make Contact (1984),0.0625,0.095833,0.033333,0.083333,0.2,0.0375,0.0,0.141667,0.0625,0.0,0.033333,0.0125,0.0375,0.0125,0.0,0.050000,0.045833,0.05,0.020833,0.020833,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,1.00,0.00,0.0,0.0
1,288,40819,4.0,Walk the Line (2005),0.0625,0.095833,0.033333,0.083333,0.2,0.0375,0.0,0.141667,0.0625,0.0,0.033333,0.0125,0.0375,0.0125,0.0,0.050000,0.045833,0.05,0.020833,0.020833,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.333333,0.00,0.00,0.0,0.0
2,288,780,1.0,Independence Day (a.k.a. ID4) (1996),0.0625,0.095833,0.033333,0.083333,0.2,0.0375,0.0,0.141667,0.0625,0.0,0.033333,0.0125,0.0375,0.0125,0.0,0.050000,0.045833,0.05,0.020833,0.020833,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.25,0.25,0.0,0.0
3,288,1021,3.0,Angels in the Outfield (1994),0.0625,0.095833,0.033333,0.083333,0.2,0.0375,0.0,0.141667,0.0625,0.0,0.033333,0.0125,0.0375,0.0125,0.0,0.050000,0.045833,0.05,0.020833,0.020833,0.00,0.00,0.0,0.5,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0
4,288,39292,4.0,"Good Night, and Good Luck. (2005)",0.0625,0.095833,0.033333,0.083333,0.2,0.0375,0.0,0.141667,0.0625,0.0,0.033333,0.0125,0.0375,0.0125,0.0,0.050000,0.045833,0.05,0.020833,0.020833,0.00,0.00,0.0,0.0,0.0,0.5,0.0,0.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7748,3,70946,5.0,Troll 2 (1990),0.2000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.000000,0.2000,0.0,0.200000,0.0000,0.0000,0.0000,0.0,0.000000,0.200000,0.20,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.5,0.0,0.5,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0
7749,175,47382,5.0,Step Up (2006),0.0000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.666667,0.0000,0.0,0.000000,0.0000,0.0000,0.0000,0.0,0.333333,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.500000,0.00,0.00,0.0,0.0
7750,175,44613,5.0,Take the Lead (2006),0.0000,0.000000,0.000000,0.000000,0.0,0.0000,0.0,0.666667,0.0000,0.0,0.000000,0.0000,0.0000,0.0000,0.0,0.333333,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.0
7751,516,4349,4.0,Catch-22 (1970),0.0000,0.000000,0.000000,0.000000,0.5,0.0000,0.0,0.000000,0.0000,0.0,0.000000,0.0000,0.0000,0.0000,0.0,0.000000,0.000000,0.00,0.500000,0.000000,0.00,0.00,0.0,0.0,0.5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.5,0.0


,Recommender,RMSE,MRE,TRE
0,SVRRecommender,1.01319,0.355558,0.226275


In [22]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

# Split into train_validation and test sets

shuffle = np.arange(len(ml_ratings_df))
rng.shuffle(shuffle)
shuffle = list(shuffle)

train_test_split = 0.8
split_index = int(len(ml_ratings_df) * train_test_split)

train_validation = ml_ratings_df.iloc[shuffle[:split_index]].loc[:, ['user_id', 'item_id', 'rating']]
test = ml_ratings_df.iloc[shuffle[split_index:]].loc[:, ['user_id', 'item_id', 'rating']]

# Tune

def loss(tuned_params):
    svr_recommender = SVRRecommender(kernel='rbf', c=tuned_params['C'], epsilon=tuned_params['epsilon'])
    rmse, mre, tre = evaluate_train_test_split_explicit(
        svr_recommender, train_validation, ml_movies_df, seed=seed)
    return rmse

# Define the search space for hyperopt
# Write your code here


succeded = False
n_tries = 3
t = 0
while not succeded and t < n_tries:
    try:
        trials = Trials()
        best_param_set = fmin(loss, space=space, algo=tpe.suggest, max_evals=100, show_progressbar=True, trials=trials)
        succeded = True
    except:
        t += 1
        traceback.print_exc()

# Best params

print("C = {}".format(best_param_set['C']))
print("epsilon = {}".format(best_param_set['epsilon']))
    
# Test

C = 0.8078294657404328
epsilon = 0.3898720771476633

svr_recommender = SVRRecommender(C=best_param_set['C'], epsilon=best_param_set['epsilon'])

results = [['SVRRecommender'] + list(evaluate_train_test_split_explicit(
    svr_recommender, {'train': train_validation, 'test': test}, ml_movies_df, seed=seed))]

results = pd.DataFrame(results, 
                       columns=['Recommender', 'RMSE', 'MRE', 'TRE'])

display(HTML(results.to_html()))

ModuleNotFoundError: No module named 'past'

**Task 7.** Gather the results for LinearRegressionRecommender, SVRRecommender, TFIDFRecommender, MostPopularRecommender, HighestRatedRecommender, RandomRecommender from the evaluate_train_test_split_implicit method and print them as a single DataFrame.

In [19]:
most_popular_recommender = MostPopularRecommender()
highest_rated_recommender = HighestRatedRecommender()
random_recommender = RandomRecommender()
lr_recommender = LinearRegressionRecommender()
svr_recommender = SVRRecommender()
tfidf_recommender = TFIDFRecommender()
recommenders = [lr_recommender, svr_recommender, tfidf_recommender, most_popular_recommender, highest_rated_recommender, random_recommender]

all_results = []

t0 = time.time()

for recommender in recommenders:
    results = [[type(recommender).__name__] + list(evaluate_train_test_split_implicit(
        recommender, ml_ratings_df, ml_movies_df))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])
    all_results.append(results)

#     display(results)
    
all_results = pd.concat(all_results).reset_index(drop=True)
print(all_results)

NameError: name 'TFIDFRecommender' is not defined